In [1]:
import yfinance as yf
import pandas as pd

# Fetch historical stock data using yfinance
def fetch_data(stock_symbol):
    # Define the stock symbol and the period for which we want the data
    period = "1d"  # Last 7 days
    interval = "1m"  # 1-minute intervals

    # Use yfinance to download the data
    stock_data = yf.download(stock_symbol, period=period, interval=interval)
    return stock_data

# Process the data to create features and labels
def prepare_data(stock_data, look_forward=10):
    # Calculate the future price after `look_forward` minutes (our label)
    stock_data['Future_Price'] = stock_data['Close'].shift(-look_forward)

    # Drop the last `look_forward` rows since they don't have a future price
    stock_data = stock_data[:-look_forward]

    return stock_data

# Replace 'AAPL' with the symbol of the stock you are interested in
stock_symbol = 'AAPL'
raw_data = fetch_data(stock_symbol)
prepared_data = prepare_data(raw_data)

[*********************100%%**********************]  1 of 1 completed


In [4]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 9.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [5]:
from sklearn.preprocessing import MinMaxScaler

def normalize_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[['Close', 'Volume', 'Future_Price']] = scaler.fit_transform(data[['Close', 'Volume', 'Future_Price']])
    return data, scaler

prepared_data, scaler = normalize_data(prepared_data)

/var/folders/cx/8v11zwh97cxgdgt_ph13_56r0000gn/T/ipykernel_69644/2196208453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Close', 'Volume', 'Future_Price']] = scaler.fit_transform(data[['Close', 'Volume', 'Future_Price']])


In [6]:
def split_data(data, test_size=0.2):
    # Calculate the index to split the data
    split_idx = int(len(data) * (1 - test_size))
    train = data[:split_idx]
    test = data[split_idx:]
    
    return train, test

train_data, test_data = split_data(prepared_data)

In [8]:
import numpy as np

def create_sequences(data, sequence_length=60):
    xs = []
    ys = []
    for i in range(len(data) - sequence_length):
        x = data[['Close', 'Volume']].iloc[i:(i + sequence_length)].values
        y = data['Future_Price'].iloc[i + sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 60  # number of minutes to look back
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = build_model((X_train.shape[1], X_train.shape[2]))

2024-03-15 01:58:25.716448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 01:58:30.104785: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-15 01:58:30.104848: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
history = model.fit(
    X_train, y_train, 
    epochs=20, 
    batch_size=32, 
    validation_data=(X_test, y_test)
)

Epoch 1/20


2024-03-15 01:58:41.507843: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 01:58:41.907295: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
2024-03-15 01:58:42.386227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 01:58:43.476508: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 01:58:44.609899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 01:58:45.072848: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


66/66 [==============================] - ETA: 0s - loss: 0.0294

2024-03-15 01:58:53.820978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 01:58:54.046706: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-15 01:58:54.224151: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


66/66 [==============================] - 19s 163ms/step - loss: 0.0294 - val_loss: 0.0058
Epoch 2/20
66/66 [==============================] - 7s 108ms/step - loss: 0.0067 - val_loss: 0.0061
Epoch 3/20
66/66 [==============================] - 7s 107ms/step - loss: 0.0060 - val_loss: 0.0049
Epoch 4/20
66/66 [==============================] - 7s 111ms/step - loss: 0.0057 - val_loss: 0.0049
Epoch 5/20
66/66 [==============================] - 7s 108ms/step - loss: 0.0053 - val_loss: 0.0049
Epoch 6/20
66/66 [==============================] - 7s 111ms/step - loss: 0.0055 - val_loss: 0.0049
Epoch 7/20
66/66 [==============================] - 7s 110ms/step - loss: 0.0053 - val_loss: 0.0080
Epoch 8/20
66/66 [==============================] - 7s 108ms/step - loss: 0.0055 - val_loss: 0.0047
Epoch 9/20
66/66 [==============================] - 7s 113ms/step - loss: 0.0051 - val_loss: 0.0044
Epoch 10/20
66/66 [==============================] - 7s 105ms/step - loss: 0.0048 - val_loss: 0.0056
Epoch 11/

In [15]:
# Evaluate the model
model.evaluate(X_test, y_test)

16/16 [==============================] - 1s 38ms/step - loss: 0.0044


0.004379523452371359

In [17]:
import numpy as np
import yfinance as yf

# Define your stock symbol and fetch data
stock_symbol = 'AAPL'
period = '60m'  # Lookback period to match your training sequence length
interval = '1m'
new_data = yf.download(stock_symbol, period=period, interval=interval)

# Assume sequence_length is defined (same as your training data)
sequence_length = 60

# Prepare the latest data for prediction
last_sequence = new_data[['Close', 'Volume']].tail(sequence_length).values

# If your scaler was initially fitted with a dummy 'Future_Price', add a dummy column to match the shape
dummy_future_price = np.zeros((last_sequence.shape[0], 1))  # Create a column of zeros
last_sequence_with_dummy = np.hstack((last_sequence, dummy_future_price))  # Add this column to your sequence

# Scale the data
last_sequence_scaled = scaler.transform(last_sequence_with_dummy)

# Remove the dummy 'Future_Price' column after scaling
last_sequence_scaled = last_sequence_scaled[:, :-1]  # Remove the dummy column, keeping only 'Close' and 'Volume'

# Reshape data for the model
last_sequence_scaled = np.expand_dims(last_sequence_scaled, axis=0)  # Add the batch dimension

# Predict using the model
predicted_price_scaled = model.predict(last_sequence_scaled)

# If necessary, reverse scaling on predicted price
# Prepare a dummy array for reverse scaling process
dummy_array = np.zeros((1, len(scaler.scale_)))  # Create a dummy array with the correct shape
dummy_array[:, -1] = predicted_price_scaled.squeeze()  # Place the predicted price in the correct column

# Reverse the scaling
predicted_price = scaler.inverse_transform(dummy_array)[:, -1]  # Extract the price from the scaled array

# Output the predicted price
print("Predicted price:", predicted_price[0])

[*********************100%%**********************]  1 of 1 completed

1/1 [==============================] - 0s 51ms/step
Predicted price: 172.96384414888144



/Users/hbot07/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
